In [2]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
load_dotenv()


True

In [5]:
api_key=os.getenv("AZURE_OPENAI_API_KEY")
api_base=os.getenv("AZURE_OPENAI_API_BASE")
api_version=os.getenv("AZURE_OPENAI_API_VERSION")
deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

In [7]:
client = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=api_base,
    api_version=api_version,
)

In [25]:
import requests

url = "https://weather-api167.p.rapidapi.com/api/weather/forecast"

#querystring = {"place":"London,GB","cnt":"3","units":"standard","type":"three_hour","mode":"json","lang":"en"}
def get_weather_forecast(place):
    
	headers = {
		"x-rapidapi-key": "80d7af33d9msh609e1f9303a6028p168c98jsn7a62ffe84df7",
		"x-rapidapi-host": "weather-api167.p.rapidapi.com",
		"Accept": "application/json"
	}
	querystring = {"place": place, "cnt": "3", "units": "standard", "type": "three_hour", "mode": "json", "lang": "en"}

	response = requests.get(url, headers=headers, params=querystring)

	return response.json()


In [30]:
weather_result=get_weather_forecast("trichy")

In [31]:
weather_result

{'cod': '200',
 'message': 0,
 'cnt': 3,
 'list': [{'dt': 1751209200,
   'summery': 'light rain, 306.17K, feels like 309.2K, winds 3.28 meter/sec from West, 49% humidity',
   'main': {'temprature': 306.17,
    'temprature_feels_like': 309.2,
    'temprature_min': 304.23,
    'temprature_max': 306.17,
    'temprature_unit': 'K',
    'pressure': 1007,
    'sea_level_pressure': 1007,
    'ground_level_pressure': 997,
    'pressure_unit': 'hPa',
    'humidity': 49,
    'humidity_unit': '%',
    'temp_kf': 1.94},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': 'https://openweathermap.org/img/wn/10n@2x.png'}],
   'clouds': {'cloudiness': 40, 'unit': '%'},
   'wind': {'speed': 3.28,
    'degrees': 268,
    'direction': 'West',
    'gust_speed': 6.8,
    'speed_unit': 'meter/sec'},
   'visibility_distance': 10000,
   'visibility_unit': 'm',
   'probability_of_precipitation': 57.99999999999999,
   'probability_of_precipitation_unit': '%',
   'rain'

In [41]:

def call_llm(query,function):
    
    result=client.chat.completions.create(
        model=deployment_name,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that provides weather information."
            },
            {
                "role": "user",
                "content": {query}
            }
        ],
        max_tokens=2048
        ).choices[0].message.content.strip()
    
    return result

## Defining function 

In [62]:
tools = [
    {
        "type": "function",
        "function": {  # <-- Nest the function definition here
            "name": "get_weather_forecast",
            "description": "Get current weather for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. Bogotá, Colombia"
                    }
                },
                "required": ["location"],
                "additionalProperties": False
            }
        }
    }
]


In [93]:
def call_llm(query):
    result = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "user", "content": query}
        ],
        max_tokens=2048,
        tools=tools,
        tool_choice="auto",
    )
    
    
    message = result.choices[0].message
    

    if message.content:
        # Model replied directly
        return message.content
    
    # Check for function call/tool call
    if hasattr(message, "tool_calls") and message.tool_calls:
        # For OpenAI: message.tool_calls is a list
        function_call = message.tool_calls[0]
        function_name = function_call.function.name
        function_args = function_call.function.arguments
        return f"Function call requested: {function_name}({function_args})",message
    
    # For older OpenAI API, it may be message.function_call
    if hasattr(message, "function_call") and message.function_call:
        return f"Function call requested: {message.function_call}"
    
    return "No response content or function call found."


In [94]:
res,message=call_llm("What is the weather forecast for Trichy?")

In [95]:
res

'Function call requested: get_weather_forecast({"location":"Trichy, India"})'

In [96]:
message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lZmVj8UWEFhI5J1GVScBq8KB', function=Function(arguments='{"location":"Trichy, India"}', name='get_weather_forecast'), type='function')])

In [99]:
import json
tool_call = message.tool_calls[0] if hasattr(message, "tool_calls") and message.tool_calls else message.function_call
print(tool_call)
args = json.loads(tool_call.function.arguments)
function_name=tool_call.function.name
print(function_name)
print(args['location'])
result = get_weather_forecast(args["location"])
print(result)

ChatCompletionMessageToolCall(id='call_lZmVj8UWEFhI5J1GVScBq8KB', function=Function(arguments='{"location":"Trichy, India"}', name='get_weather_forecast'), type='function')
get_weather_forecast
Trichy, India
{'cod': '200', 'message': 0, 'cnt': 3, 'list': [{'dt': 1751252400, 'summery': 'scattered clouds, 303.17K, feels like 308.24K, winds 8.09 meter/sec from West-Northwest, 70% humidity', 'main': {'temprature': 303.17, 'temprature_feels_like': 308.24, 'temprature_min': 303.17, 'temprature_max': 303.32, 'temprature_unit': 'K', 'pressure': 1007, 'sea_level_pressure': 1007, 'ground_level_pressure': 998, 'pressure_unit': 'hPa', 'humidity': 70, 'humidity_unit': '%', 'temp_kf': -0.15}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': 'https://openweathermap.org/img/wn/03d@2x.png'}], 'clouds': {'cloudiness': 40, 'unit': '%'}, 'wind': {'speed': 8.09, 'degrees': 286, 'direction': 'West-Northwest', 'gust_speed': 9.61, 'speed_unit': 'meter/sec'}, 'visibility_dis

### Must send the info on the  function call and function response to gpt

In [123]:
messages = [
    {"role": "user", "content": "What is the weather forecast for Trichy?"},
    {
        "role": "assistant",
        "content": None,
        "tool_calls": [
            {
                "id": "call_lZmVj8UWEFhI5J1GVScBq8KB",
                "type": "function",
                "function": {
                    "name": "get_weather_forecast",
                    "arguments": '{"location": "Trichy, India"}'
                }
            }
        ]
    },
    {
        "role": "function",
        "name": "get_weather_forecast",
        "content": json.dumps(result),
        "tool_call_id": "call_lZmVj8UWEFhI5J1GVScBq8KB"# e.g., the weather data as a string/JSON
    }
]


In [121]:
messages

[{'role': 'user', 'content': 'What is the weather forecast for Trichy?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_lZmVj8UWEFhI5J1GVScBq8KB',
    'type': 'function',
    'function': {'name': 'get_weather_forecast',
     'arguments': '{"location": "Trichy, India"}'}}]},
 {'role': 'function',
  'name': 'get_weather_forecast',
  'content': {'cod': '200',
   'message': 0,
   'cnt': 3,
   'list': [{'dt': 1751252400,
     'summery': 'scattered clouds, 303.17K, feels like 308.24K, winds 8.09 meter/sec from West-Northwest, 70% humidity',
     'main': {'temprature': 303.17,
      'temprature_feels_like': 308.24,
      'temprature_min': 303.17,
      'temprature_max': 303.32,
      'temprature_unit': 'K',
      'pressure': 1007,
      'sea_level_pressure': 1007,
      'ground_level_pressure': 998,
      'pressure_unit': 'hPa',
      'humidity': 70,
      'humidity_unit': '%',
      'temp_kf': -0.15},
     'weather': [{'id': 802,
       'main': 'Clouds',
       'des

In [124]:
res_2=client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=2048,
    tools=tools
)

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_lZmVj8UWEFhI5J1GVScBq8KB", 'type': 'invalid_request_error', 'param': 'messages.[2].role', 'code': None}}